In [3]:
import datetime

In [5]:
import os
pwd = os.path.abspath('.')
os.chdir(os.path.join(pwd, '../src/'))

  ### Get Index data for date range

In [9]:
from datareader.data_reader import NseIndicesReader
reader = NseIndicesReader()

from datareader.data_reader import DateRangeDataReader
daterange_reader = DateRangeDataReader(reader)

from_date = datetime.date(2023, 1, 1)
to_date = datetime.date.today() + datetime.timedelta(days=-1)
result = daterange_reader.read(from_date = from_date, to_date = to_date)

### Calculation pipeline for RSI
Below example demonstrates calculating RSI using the calculation pipeline. The datepart calculation is pre-requisite for RSI calculation.

In [13]:
# import classes & setup options
from dataprocess.data_processor import HistoricalDataProcessor, MultiDataCalculationPipelines, CalculationPipelineBuilder, HistoricalDataProcessOptions
from calculations.base import DatePartsCalculationWorker
options = HistoricalDataProcessOptions()
options.include_monthly_data = False
options.include_annual_data = False
histDataProcessor = HistoricalDataProcessor(options)

# Fetch the data
year_start = datetime.date(2023, 1, 1)
to_date = datetime.date.today() + datetime.timedelta(days=-1)
result = histDataProcessor.process(reader, {'from_date': year_start, 'to_date': to_date})

# Prepare calculation pipeline
pipelines = MultiDataCalculationPipelines()
pipelines.set_item('date_parts', CalculationPipelineBuilder.create_pipeline_for_worker(DatePartsCalculationWorker()))
pipelines.set_item('rsi', CalculationPipelineBuilder.create_rsi_calculation_pipeline())
histDataProcessor.set_calculation_pipelines(pipelines)

# Run the pipeline and get data
histDataProcessor.run_calculation_pipelines()
result.get_daily_data()

Started to read data
get_data took 1 seconds
get_manual_data took 0 seconds
Started basic calculation
add_basic_calc took 0 seconds
process took 1 seconds
DatePartsCalculationWorker took 0 seconds
RsiCalculationWorker took 1 seconds
ValueCrossedAboveFlagWorker took 0 seconds
ValueCrossedBelowFlagWorker took 0 seconds


,Identifier,Index Date,Open,High,Low,Close,Points Change,Change(%),Volume,Turnover (Rs. Cr.),...,Div Yield,Date,Growth,Year,MonthNo,Month,Day,Rsi,RsiCrossedAbove,RsiCrossedBelow
0,INDIA VIX,02-01-2023,14.8675,15.4525,14.5675,14.69,-0.18,-1.23,-,-,...,-,2023-01-02,0.000000,2023,1,Jan,Monday,NaN,False,False
1,INDIA VIX,03-01-2023,14.685,14.99,14.305,14.39,-0.30,-2.04,-,-,...,-,2023-01-03,-2.042206,2023,1,Jan,Tuesday,NaN,False,False
2,INDIA VIX,04-01-2023,14.385,15.6825,14.33,15.20,0.81,5.65,-,-,...,-,2023-01-04,3.471749,2023,1,Jan,Wednesday,NaN,False,False
3,INDIA VIX,05-01-2023,15.1975,15.6,14.8125,14.98,-0.22,-1.41,-,-,...,-,2023-01-05,1.974132,2023,1,Jan,Thursday,NaN,False,False
4,INDIA VIX,06-01-2023,14.9825,15.4375,14.6225,15.03,0.04,.28,-,-,...,-,2023-01-06,2.314500,2023,1,Jan,Friday,NaN,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27288,NIFTY500 VALUE 50,01-01-2024,-,-,-,11457.93,93.57,.82,705534086,13278.45,...,4.06,2024-01-01,59.181028,2024,1,Jan,Monday,75.592678,False,False
27289,NIFTY500 VALUE 50,02-01-2024,-,-,-,11473.96,16.02,.14,728715398,14950.91,...,4.05,2024-01-02,59.403727,2024,1,Jan,Tuesday,75.859518,True,False
27290,NIFTY500 VALUE 50,03-01-2024,-,-,-,11445.22,-28.74,-.25,731243686,14596.62,...,4.06,2024-01-03,59.004453,2024,1,Jan,Wednesday,74.291295,False,False
27291,NIFTY500 VALUE 50,04-01-2024,-,-,-,11540.30,95.08,.83,723689710,15219.37,...,4.03,2024-01-04,60.325366,2024,1,Jan,Thursday,76.054901,False,False


### A real use case: Understand the affect of RSI and Stochastic RSI on price
In this use case, understand the affect of RSI and Stochastic RSI on price

##Preparing the data
Calculate RSI and Stochastic RSI for each day.
Add a flag for whenever the RSI crosses the control limits (eg: above 75 and below 30)
Calculate the highest and lowest price change in the next 10 trading sessions.

##Analyse
We will find the average for highest price change and lowest price change whenever the RSI crosses the threshold.

In [14]:
# import classes
from datareader.data_reader import BhavCopyReader
reader = BhavCopyReader()

from datareader.data_reader import MultiDatesDataReader, DateRangeDataReader
daterange_reader = DateRangeDataReader(reader)

# Fetch the data
year_start = datetime.date(2023, 1, 1)
to_date = datetime.date.today() + datetime.timedelta(days=-1)
result = histDataProcessor.process(reader, {'from_date': year_start, 'to_date': to_date})

Started to read data
Reading data from 2023-10-27 to 2024-01-05
https://archives.nseindia.com/content/historical/EQUITIES/2023/OCT/cm30OCT2023bhav.csv.zip
https://archives.nseindia.com/content/historical/EQUITIES/2023/OCT/cm31OCT2023bhav.csv.zip
https://archives.nseindia.com/content/historical/EQUITIES/2023/NOV/cm01NOV2023bhav.csv.zip
https://archives.nseindia.com/content/historical/EQUITIES/2023/NOV/cm02NOV2023bhav.csv.zip
https://archives.nseindia.com/content/historical/EQUITIES/2023/NOV/cm03NOV2023bhav.csv.zip
https://archives.nseindia.com/content/historical/EQUITIES/2023/NOV/cm06NOV2023bhav.csv.zip
https://archives.nseindia.com/content/historical/EQUITIES/2023/NOV/cm07NOV2023bhav.csv.zip
https://archives.nseindia.com/content/historical/EQUITIES/2023/NOV/cm08NOV2023bhav.csv.zip
https://archives.nseindia.com/content/historical/EQUITIES/2023/NOV/cm09NOV2023bhav.csv.zip
https://archives.nseindia.com/content/historical/EQUITIES/2023/NOV/cm10NOV2023bhav.csv.zip
https://archives.nseindia.

In [27]:
# prepare calculation pipeline
periods = [1, 7, 15, 30, 45]

pipelines = MultiDataCalculationPipelines()
pipelines.set_item('date_parts', CalculationPipelineBuilder.create_pipeline_for_worker(DatePartsCalculationWorker()))
pipelines.set_item('rsi', CalculationPipelineBuilder.create_rsi_calculation_pipeline(crossing_above_flag_value = 75, crossing_below_flag_value = 30, window = 14))
pipelines.set_item('stoch_rsi', CalculationPipelineBuilder.create_stoch_rsi_calculation_pipeline(crossing_above_flag_value = 80, crossing_below_flag_value = 20, window = 14))
pipelines.set_item('foward_looking_fall', CalculationPipelineBuilder.create_forward_looking_price_fall_pipeline(periods))
pipelines.set_item('foward_looking_rise', CalculationPipelineBuilder.create_forward_looking_price_rise_pipeline(periods))
histDataProcessor.set_calculation_pipelines(pipelines=pipelines)

# run the pipeline and show results
histDataProcessor.run_calculation_pipelines()

daily_data = result.get_daily_data()

# Import constants so its easier to refer to column names
from core.column_definition import BaseColumns, CalculatedColumns

# get names of fwd looking price column names. Since, these column names are auto-generated there no constants for them
fwd_looking_price_fall_cols, fwd_looking_price_rise_cols = [x for x in daily_data.columns if 'HighestPercFallInNext' in x], \
    [x for x in daily_data.columns if 'HighestPercRiseInNext' in x]

# analyse the median price change % for highest price fall whenever the RSI crosses above
daily_data[
  (daily_data[CalculatedColumns.RsiCrossedAbove])
][fwd_looking_price_fall_cols].median()

# analyse the median price change % for highest price rise whenever the RSI crosses below
daily_data[
  (daily_data[CalculatedColumns.RsiCrossedAbove])
][fwd_looking_price_rise_cols].median()